### State Farm Distracted Driving: Classifying images based on driver safety

See Also: https://www.kaggle.com/praveenmaripeti/state-farm-distracted-driver-detection-with-keras for NN implemntation with tensorflow

Distracted driving causes a lot of accidents and is 100% preventable. Machine learning algorithms can gage driver safety using 2D dashboard camera images of drivers. The goal is to classify these images based on driver's behavior (cell phone, texting, etc). 

10 behaviors are classfied as follows:

In [6]:
classes = ['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']
#['c'+str(x) for x in range(10)]

class_def = {'c0': 'safe driving',
'c1': 'texting - right',
'c2': 'talking on the phone - right',
'c3': 'texting - left',
'c4': 'talking on the phone - left',
'c5': 'operating the radio',
'c6': 'drinking',
'c7': 'reaching behind',
'c8': 'hair and makeup',
'c9': 'talking to passenger'}

In [ ]:
# Import Relevant Libaries
import os, shutil
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import random

#SVM
from sklearn.svm import SVC


In [7]:
# Some parameters
#overwrite image data? 
data_overwrite = False